In [ ]:
from bs4 import BeautifulSoup
import os
import nltk
import re
import requests
import numpy as np
from nltk.corpus import stopwords
from collections import Counter,defaultdict
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import snowball
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from flask import Flask,request,render_template

In [ ]:
#Parsing the webapge and obtaining the urls
url='https://www.memphis.edu/datascience/'
all_urls=['https://www.memphis.edu/datascience/']
res=requests.get(url)
soup=BeautifulSoup(res.text)

for tag in soup.find_all('a',href=True):
    link=tag.get('href').strip()
    if(link.startswith('http')):
        all_urls.append(link)

all_urls=list(set(all_urls))



In [ ]:
# storing the text content scraped from webpage into text files
def get_text_from_webpage(hyperlink):
    res=requests.get(hyperlink)
    soup=BeautifulSoup(res.text)
    text_content=re.sub(r'\s+',' ',soup.get_text(separator=' ',strip=True))
    return text_content
    
path_to_store_raw_files='./scraped input files/'
for url_index in range(len(all_urls)):
    file_content=get_text_from_webpage(all_urls[url_index])
    with open(path_to_store_raw_files+'Raw_document_'+str(url_index)+'.txt', 'w',encoding='utf-8') as raw_file:
        raw_file.write(file_content)

In [ ]:
#Processing the text files by removing stop words, punctuation, numbers and CAPITAL WORDS and special characters
stop_words = stopwords.words('english')
stopwords_dict = Counter(stop_words)
path_to_store_raw_files='./scraped input files/'
def process_files(file):
    processed_text=''
    with open(path_to_store_raw_files+file,'r',encoding='utf-8') as handler:
        modify_file=handler.read()
        pre_processed_text=re.sub(r'[\d+\W]',' ',modify_file)
        pre_processed_text=re.sub(r'[A-Z]+(?![a-z])', '', pre_processed_text)
        pre_processed_text=re.sub(r'\s+',' ',pre_processed_text)
        pre_processed_words=list(pre_processed_text.split())
        processed_text= ' '.join([word.lower() for word in pre_processed_words if word.lower() not in stopwords_dict])  
    return processed_text

path_to_store_processed_files='./processed files/'
for ip_file in os.listdir('./scraped input files/'):
    doc_index=int(''.join(filter(str.isdigit,ip_file)))
    with open(path_to_store_processed_files+'Processed_document_'+str(doc_index),'w',encoding='utf-8') as processed_file:
        processed_file.write(process_files(ip_file))

In [ ]:
#generate the inverted index based on the documents scraped from web
path_to_store_processed_files='./processed files/'
corpus={}
stemmer = snowball.SnowballStemmer('english')
inverted_index = defaultdict(set)
for proc_file in os.listdir('./processed files/'):
    doc_index=int(''.join(filter(str.isdigit,proc_file)))
    with open(path_to_store_processed_files+proc_file,'r',encoding='utf-8') as add_to_corpus:
        string_text=add_to_corpus.read()
        corpus[doc_index]=string_text

for doc_id,doc_text in corpus.items():
    for sent in sent_tokenize(doc_text):
        for word in word_tokenize(sent):
            word_stem = stemmer.stem(word)
            inverted_index[word_stem].add(doc_id)

In [ ]:
#Search the inverted index to get the relevenat documents for the given query
stop_words = stopwords.words('english')
stopwords_dict = Counter(stop_words)
def search_documents(query):
    matched_documents = set()
    matched_documents_all = set()
    for word in word_tokenize(query):
        word_lower = word.lower()
        if word_lower not in stopwords_dict:
            word_stem = stemmer.stem(word_lower)
            matches = inverted_index.get(word_stem)
            if matches:
                # The operator |= is a short hand for set union to get matching from either of words in query
                matched_documents_all |= matches
                matched_documents = matches
            
    return matched_documents,matched_documents_all

In [ ]:
#Rank documents based on cosine similarity
'''docs_tofind_similarity=[]
urls_of_matched_docs=[]
string_to_search=input('Enter a query to search : ')
for index in list(search_documents(string_to_search)):
    docs_tofind_similarity.append(corpus[index])
    urls_of_matched_docs.append(all_urls[index])
    
docs_tofind_similarity.append(string_to_search)'''

def vectorize(docs_related):
    tf_idf = TfidfVectorizer().fit_transform(docs_related)
    obtained=cosine_similarity(tf_idf[-1:],tf_idf) #as this is a list of list
    cosine=obtained[0][:-1]
    #sorting the cosine values and retrieving the index of each value after sort. Thus max similarity comes first and on..
    all_documents_sorted_index=np.argsort(cosine)[::-1].tolist()
    #Assuming to retrieve documents with similarity greater than 30%
    docs_greater_than_50_percent=np.argwhere(np.array(cosine) >= 0.05).reshape(1,np.argwhere(np.array(cosine)>=0.05).size).tolist()[0]
    
    return all_documents_sorted_index,docs_greater_than_50_percent


'''
Precision = total no. of docs retrieved that are relevant
            ---------------------------------------------
            total no. of docs that are retrieved
        
Recall =    total no. of docs retrieved that are relevant
            ---------------------------------------------
            total no. of docs relevant documents in database
'''



In [ ]:
# A web interface to perform the query search
app = Flask(__name__)

@app.route('/')
def welcome_page():
    return render_template('starting_page.html')

@app.route('/',methods=['POST'])
def landing_page():
    string_to_search=request.form.get('inputs')
    
    docs_tofind_similarity=[]
    urls_of_matched_docs=[]
    relevant_documents,retrieved_documents=search_documents(string_to_search)
      
    for index in list(relevant_documents):
        docs_tofind_similarity.append(corpus[index])
        urls_of_matched_docs.append(all_urls[index])
        
    docs_tofind_similarity.append(string_to_search)
    
    all_retrieved_docs,docs_to_display= vectorize(docs_tofind_similarity)

    links_to_files=[urls_of_matched_docs[all_retrieved_docs[x]] for x in range(len(docs_to_display))]
        
    if(len(docs_to_display)==0):
        return render_template('landing_page.html',length=1,link=['No Matching Found'],precision=0,recall=0,f1_score=0)
    
    precision= round(len(docs_to_display)/len(all_retrieved_docs),3)
    recall= round(len(docs_to_display)/len(retrieved_documents),3)
    f1_score=round(2*(precision*recall)/(precision+recall),3)
    
    
    return render_template('landing_page.html',length=len(docs_to_display),link=links_to_files,precision=precision,recall=recall,f1_score=f1_score)
                              
if __name__=='__main__':
   app.run()                    

In [ ]:
my=[None]*5

my[2]='hello'
print(my)

docs={0,123,89}
print(list(docs))

In [ ]:
import numpy as np
a=[0.54,0.678,0.98,1.09,0.87]

print(np.argsort(a)[::-1].tolist())

np.argwhere(np.array(a) > 0.64).reshape(1,np.argwhere(np.array(a)>0.64).size).tolist()[0]

In [ ]:
my=['Cat speak meow','dog speak woof', 'i speak language','cat and dog are animals','I and harry a dog play together','namaste dear']

def vectorize(docs_related):
    tf_idf = TfidfVectorizer().fit_transform(docs_related)
    obtained=cosine_similarity(tf_idf[-1:],tf_idf) #as this is a list of list
    cosine=obtained[0][:-1]
    print(cosine)
    #sorting the cosine values and retrieving the index of each value after sort. Thus max similarity comes first and on..
    all_documents_sorted_index=np.argsort(cosine)[::-1].tolist()
    #Assuming to retrieve documents with similarity greater than 30%
    docs_greater_than_50_percent=np.argwhere(np.array(cosine) >= 0.1).reshape(1,np.argwhere(np.array(cosine)>=0.1).size).tolist()[0]
    
    return all_documents_sorted_index,docs_greater_than_50_percent

print(vectorize(my))